## Implementation 1
### Feature Engineering
#### 1. SSC+ 

#### 2. SSC Inner

#### 3. SSC 

#### 4. Preferential Attachment Score 

#### 5. Total Friends


| Feature | Description | Note/Implication |
|:--------|:------------|:-----|
| SSC+   |    |      |
| SSC Inner      |            |      |
| SSC       |             |      |
| Preferential Attachment Score         |             |      |
| Total Friends       |             |      |
|         |             |      |

# Data 
#### File descriptions

- train.txt - the training graph adjacency lists (tab delimited; line per source node followed by sink neighbours; no header line)  
- test-public.txt - the test edge IDs (tab delimited; edge ID, source node, sink node; with header line)  
- sample.csv - a sample submission file in the correct format  

### Importing All Data 

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [2]:
G =  nx.read_adjlist('data/train.txt', delimiter='\t', create_using=nx.DiGraph(), nodetype=int)

## Sampling the Data
For faster computational performance.

In [ ]:
from sampler import random_sampler

num_sample = 2400000
sample_data = random_sampler('data/train.txt', num_sample)
G_sample =  nx.read_adjlist(sample_data, delimiter='\t', create_using=nx.DiGraph() )

### Getting Sample Statistics

In [ ]:
from stats import get_network_statistics
get_network_statistics(G)

## Features

In [3]:

features = pd.read_csv('data_features.csv')


In [ ]:

#features['SCC'] = 0 # features.apply(lambda row: SCC(row.source, row.target), axis=1)
#features['SCC_p'] = 0 # features.apply(lambda row: SCC_p(row.source, row.target), axis=1)
features['pref_attachment'] = features.apply(lambda row: pref_attachment(row.source, row.target), axis=1)
features['total_friends'] = features.apply(lambda row: total_friends(G, row.source, row.target), axis=1)
features['common_friends'] = features.apply(lambda row: common_friends(row.source, row.target), axis=1)

features['SCC_p'] = 0 # features.apply(lambda row: SCC_p(row.source, row.target), axis=1)
features['jacard_coef'] = features.apply(lambda row: jacard_coef(row.source, row.target), axis=1)
features['total_friends'] = 0 # features.apply(lambda row: total_friends(row.source, row.target), axis=1)


features.to_csv('data_features.csv')

In [4]:
features_sample = features.loc[:5,]
import time

t0= time.clock()
features_sample['pref_attachment'] =  pref_attach(features_sample['source'], features_sample['target'])
t1 = time.clock() 
print("Time elapsed: ", t1 - t0)
t0= time.clock()
features_sample['total_friends'] = features_sample.apply(lambda row: total_friends(row.source, row.target), axis=1)
t1 = time.clock() 
print("Time elapsed: ", t1 - t0)
t0= time.clock()
features_sample['common_friends'] = features_sample.apply(lambda row: common_friends(row.source, row.target), axis=1)
t1 = time.clock()
print("Time elapsed: ", t1 - t0)




NameError: name 'pref_attach' is not defined

In [5]:

# Defining intersection and union of lists.
def intersection(a, b):
    return list(set(a) & set(b))
def union(a, b):
    return list(set(a) | set(b))

# Trying to be consistent with notation from the paper to avoid confusion.
# Neighborhood functions
def Gamma(u):
    return union(G.successors(u),G.predecessors(u))
def Gamma_in(u):
    return G.predecessors(u)
def Gamma_out(u):
    return G.successors(u)
def Gamma_plus(u):
    return union(Gamma(u),{u})    


# SUBGRAPH FUNCTIONS
# I represent these in subgraph form rather than edge list, as specified in the
# paper. I think it will be more efficient this way, as we can use the nx.compose
# function and others.

# nh subgraph for vertex
def nh_subgraph_vertex(u):
    return G.subgraph(Gamma(u))

def nh_subgraph_vertex_plus(u):
    return G.subgraph(Gamma_plus(u))

# nh subgraph for edge
def nh_subgraph_edge(u,v):
    return G.subgraph(union(Gamma(u),Gamma(v)))

def nh_subgraph_edge_plus(u,v):
    return G.subgraph(union(Gamma(u),Gamma(v)))

def inner_subgraph(u,v):
    g1 = Gamma(u)
    g2 = Gamma(v)
    # some function that grabs a full edgelist
    # for some edge (a,b)
    g = nh_subgraph_edge(u,v)
    e = g.edges()
    inner = nx.DiGraph() # create empty subgraph structure
    for i in e:
        if ((i[0] in g1) and (i[1] in g2)) or ((i[1] in g1) and (i[0] in g2)):
            inner.add_edge(i[0],i[1]) # add this edge to inner
    return inner


# DEFINING FEATURES


# COMPUTATIONALLY FAST
# TOTAL FRIENDS
def total_friends(u,v):
    return len(union(Gamma(u),Gamma(v)))

# PREFERENTIAL ATTACHMENT
def pref_attach(u,v):
    return len(Gamma(u))*len(Gamma(v))

# COMMON FRIENDS
def common_friends(u,v):
    return len(intersection(Gamma(u),Gamma(v)))

# JACARD'S COEFFICIENT
def jacard_coef(u,v):
    return common_friends(u,v)/total_friends(u,v)

# FRIENDS MEASURE (slightly slower)
def friends_measure(u,v):
    counter = 0
    for i in Gamma(u):
        for j in Gamma(v):
            if (i == j) or (G.has_edge(i,j) == True) or (G.has_edge(j,i) == True):
                counter = counter + 1
    return counter


# COMPUTATIONALLY SLOW.
# scc nh subgraph
def scc_nh(u,v):
    return nx.number_strongly_connected_components(nh_subgraph_edge(u,v))  

# scc nh subgraph +
def scc_nh_plus(u,v):
    return nx.number_strongly_connected_components(nh_subgraph_edge_plus(u,v)) 

# scc inner subgraph
def scc_inner(u,v):
    return nx.number_strongly_connected_components(inner_subgraph(u,v))



In [7]:
t0= time.clock()

pref_attach(2926583,4380400 )
t1 = time.clock() 
print("Time elapsed: ", t1 - t0)

Time elapsed:  0.0001239999999995689


In [8]:
import time
features_sample = features.loc[:5,]


t0= time.clock()
features_sample['pref_attachment'] =  features_sample.apply(lambda row: pref_attach(row.source, row.target), axis=1)
t1 = time.clock() 
print("Time elapsed: ", t1 - t0)

Time elapsed:  17.948119000000005


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
UnsatisfiableError: The following specifications were found to be in conflict:
  - conda=4.0.8
  - entrypoints
Use "conda info <package>" to see the dependencies for each package.

  Found existing installation: alabaster 0.7.10
    Uninstalling alabaster-0.7.10:
      Successfully uninstalled alabaster-0.7.10
    100% |████████████████████████████████| 348kB 4.7MB/s ta 0:00:01
  Found existing installation: anaconda-project 0.8.0
    Uninstalling anaconda-project-0.8.0:
      Successfully uninstalled anaconda-project-0.8.0
    100% |████████████████████████████████| 102kB 1.2MB/s a 0:00:01
  Found existing installation: asn1crypto 0.22.0
    Uninstalling asn1crypto-0.22.0:
      Successfully uninstalled asn1crypto-0.22.0
    100% |████████████████████████████████| 174kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 215kB 2.3MB/s ta 0:00:01
  Found existing installation: astroid 1.5.3
    Uninstalling astroid-1.5.3:
      Successfully uninstalled astroid-1.5.3
    100% |████████████████████████████████| 7.7MB 1.5MB/s eta 0:00:01
  Found existing installation: astropy 2.0.2
    Uninstalling astropy-2.0.2:
      Successfully uninstalled astropy-2.0

  Found existing installation: bleach 2.0.0
    Uninstalling bleach-2.0.0:
      Successfully uninstalled bleach-2.0.0
    100% |████████████████████████████████| 16.0MB 1.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for bokeh ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/05/3e/43/95ff0bde940a0a5d86ec13c22d2a4bddc97271cd788f441a63
Successfully built bokeh
  Found existing installation: bokeh 0.12.10
    Uninstalling bokeh-0.12.10:
      Successfully uninstalled bokeh-0.12.10
    100% |████████████████████████████████| 1.4MB 4.9MB/s ta 0:00:01
  Found existing installation: boto 2.48.0
Cannot uninstall 'boto'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
    100% |████████████████████████████████| 133kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 4.6MB 2.3MB/s eta 0:00:01
  Found existing installation: botocore 1.10.82
    Uninstalling bot

    100% |████████████████████████████████| 5.3MB 1.8MB/s eta 0:00:01
  Found existing installation: Cython 0.26.1
    Uninstalling Cython-0.26.1:
Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/anaconda3/lib/python3.6/site-packages/__pycache__/cython.cpython-36.pyc'

    100% |████████████████████████████████| 450kB 3.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for cytoolz ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/88/f3/11/9817b001e59ab04889e8cffcbd9087e2e2155b9ebecfc8dd38
Successfully built cytoolz
thinc 6.10.2 has requirement cytoolz<0.9,>=0.8, but you'll have cytoolz 0.9.0.1 which is incompatible.
  Found existing installation: cytoolz 0.8.2
    Uninstalling cytoolz-0.8.2:
      Successfully uninstalled cytoolz-0.8.2
    100% |████████████████████████████████| 655kB 4.3MB/s ta 0:00:01
  Found existing installation: dask 0.15.3
    Uninstalling dask-0.15.3:
      Successfully uninstalled dask-0.15.3
    1

  Found existing installation: Jinja2 2.9.6
    Uninstalling Jinja2-2.9.6:
      Successfully uninstalled Jinja2-2.9.6
  Found existing installation: Werkzeug 0.12.2
    Uninstalling Werkzeug-0.12.2:
      Successfully uninstalled Werkzeug-0.12.2
  Found existing installation: Flask 0.12.2
    Uninstalling Flask-0.12.2:
      Successfully uninstalled Flask-0.12.2
  Found existing installation: Flask-Cors 3.0.3
    Uninstalling Flask-Cors-3.0.3:
      Successfully uninstalled Flask-Cors-3.0.3
    100% |████████████████████████████████| 3.1MB 4.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 9.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for greenlet ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/c6/54/b6/34dc665cda3b4c2ce4ab1471dbebb690b1cea04cbcfd50347a
Successfully built greenlet
  Found existing installation: greenlet 0.4.12
Cannot uninstall 'greenlet'. It is a distutils installed project and thus we cannot accurately determine which file

  Found existing installation: ipykernel 4.6.1
    Uninstalling ipykernel-4.6.1:
      Successfully uninstalled ipykernel-4.6.1
    100% |████████████████████████████████| 757kB 3.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for backcall ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/98/b0/dd/29e28ff615af3dda4c67cab719dd51357597eabff926976b45
Successfully built backcall
  Found existing installation: ipython 6.1.0
    Uninstalling ipython-6.1.0:
      Successfully uninstalled ipython-6.1.0
    100% |████████████████████████████████| 112kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 2.2MB 2.5MB/s ta 0:00:01
  Found existing installation: widgetsnbextension 3.0.2
    Uninstalling widgetsnbextension-3.0.2:
      Successfully uninstalled widgetsnbextension-3.0.2
  Found existing installation: ipywidgets 7.0.0
    Uninstalling ipywidgets-7.0.0:
      Successfully uninstalled ipywidgets-7.0.0
    100% |████████████████████████████████| 51kB 863kB

...

    100% |████████████████████████████████| 215kB 8.5MB/s eta 0:00:01
  Found existing installation: seaborn 0.8
    Uninstalling seaborn-0.8:
      Successfully uninstalled seaborn-0.8
    100% |████████████████████████████████| 573kB 4.6MB/s ta 0:00:011
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.4 which is incompatible.
  Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/anaconda3/lib/python3.6/site-packages/pkg_resources/__init__.py'
Consider using the `--user` option or check the permissions.

  Found existing installation: sortedcollections 0.5.3
    Uninstalling sortedcollections-0.5.3:
      Successfully uninstalled sortedcollections-0.5.3
  Found existing installation: sortedcontainers 1.5.7
    Uninstalling sortedcontainers-1.5.7:
      Successfully uninstalled sortedcontainers-1.5.7
    100% |█████████████

    100% |████████████████████████████████| 1.9MB 4.6MB/s ta 0:00:01
  Found existing installation: Sphinx 1.6.3
    Uninstalling Sphinx-1.6.3:
      Successfully uninstalled Sphinx-1.6.3
  Found existing installation: sphinxcontrib-websupport 1.0.1
    Uninstalling sphinxcontrib-websupport-1.0.1:
      Successfully uninstalled sphinxcontrib-websupport-1.0.1
    100% |████████████████████████████████| 2.7MB 5.5MB/s ta 0:00:01
    100% |████████████████████████████████| 81.7MB 272kB/s eta 0:00:01    55% |█████████████████▋              | 45.0MB 11.8MB/s eta 0:00:043.9MB 62.1MB/s eta 0:00:01█████████████████       | 64.1MB 47.1MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 9.4MB/s eta 0:00:01


    100% |████████████████████████████████| 61kB 18.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/04/d1/c6616dd03701e7e2073f06d5c3b41b012256e42b72561f16a7bd86dd7b43/prompt_toolkit-1.0.15-py3-none-any.whl
  Found existing installation: spyder 3.2.4
Cannot uninstall 'spyder'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
    100% |████████████████████████████████| 5.6MB 3.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for SQLAlchemy ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/2b/e7/bc/03c832ebc5c8f34a4ddab35f1bc812feb1e606d1147d6af2c9
Successfully built SQLAlchemy
  Found existing installation: SQLAlchemy 1.1.13
    Uninstalling SQLAlchemy-1.1.13:
      Successfully uninstalled SQLAlchemy-1.1.13
    100% |████████████████████████████████| 9.6MB 1.9MB/s ta 0:00:011


  Found existing installation: statsmodels 0.8.0
    Uninstalling statsmodels-0.8.0:
      Successfully uninstalled statsmodels-0.8.0
    100% |████████████████████████████████| 5.4MB 3.6MB/s eta 0:00:01
  Running setup.py bdist_wheel for sympy ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/b4/af/81/7fad05f53f1e451e8ddb4b15aa3908dc98112a9459911ebb66
Successfully built sympy
  Found existing installation: sympy 1.1.1
Cannot uninstall 'sympy'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
    100% |████████████████████████████████| 7.0MB 3.2MB/s ta 0:00:011
  Found existing installation: tables 3.4.2
    Uninstalling tables-3.4.2:
      Successfully uninstalled tables-3.4.2
    100% |████████████████████████████████| 56.1MB 346kB/s eta 0:00:014MB/s eta 0:00:01█████████████████| 55.9MB 43.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f6/cd

  Running setup.py bdist_wheel for thinc ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/f6/06/79/bfb440b8ab1eb00bc116d8e1845a032dcc144cc6933f03a49b
  Running setup.py bdist_wheel for hypothesis ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/ff/6d/23/efba0d5f717baec2323a0416fa739a6fa92da25f882a855b6b
  Running setup.py bdist_wheel for msgpack-python ... done
  Stored in directory: /Users/najla/Library/Caches/pip/wheels/f5/52/33/f49e5fca0bd22a9c0d7ab85320f2d0e6ff5fe49cec948d673a
Successfully built thinc hypothesis msgpack-python
spacy 2.0.12 has requirement thinc<6.11.0,>=6.10.3, but you'll have thinc 6.11.2 which is incompatible.
  Found existing installation: msgpack-python 0.5.6
    Uninstalling msgpack-python-0.5.6:
      Successfully uninstalled msgpack-python-0.5.6
  Found existing installation: msgpack-numpy 0.4.3.1
    Uninstalling msgpack-numpy-0.4.3.1:
      Successfully uninstalled msgpack-numpy-0.4.3.1
  Found existing installat

In [15]:
features_split = np.array_split(features, 20)

NameError: name 'np' is not defined

In [11]:
t0= time.clock()
features_sample['total_friends'] = features_sample.apply(lambda row: total_friends(row.source, row.target), axis=1)
t1 = time.clock() 
print("Time elapsed: ", t1 - t0)

Time elapsed:  6.482231999999996


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
t0= time.clock()
features_sample['common_friends'] = features_sample.apply(lambda row: common_friends(row.source, row.target), axis=1)
t1 = time.clock()
print("Time elapsed: ", t1 - t0)

Time elapsed:  6.328578000000022


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
import numpy as np
x = np.array_split(features_sample, 3)

In [28]:
x[3]

IndexError: list index out of range

In [24]:
x[1]['pref_attachment'] = x[1].apply(lambda row: pref_attach(row.source, row.target), axis=1)
x[1]['d'] = 2

In [32]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('G.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(G, f)

# Getting back the objects:
with open('G.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
       G = pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
# Getting back the objects:
with open('G.pkl', 'rb') as f: 
       G = pickle.load(f)

In [ ]:
type(G)